<h1>Weather data table</h1>

<h3>IMPORTS</h3>

In [1]:
#IMPORTS
import pandas as pd
# import sqlite3
import os
from google.cloud.sql.connector import Connector, IPTypes
import pymysql
import sqlalchemy
from zipfile import ZipFile
import numpy as np

<h3>METHODS</h3>

In [3]:
#process 1 csv file 

def extract_data(file):

    try:
        df = pd.read_csv(file, encoding='latin1', header = None, keep_default_na=False, index_col=None)
    except Exception as e:
        raise Exception("ERROR READING CSV:", file, "-- WITH ERROR:", e)   
    
    df = df.drop([0,1,2,3,4,5,6])  # Drop the first 7 rows


    try:
        df = df.iloc[:, [0, 1, 3, 5, 6, 7, 8 , 9 ]]   
    except:
        raise Exception("column names not properly formatted in file")

    row1 = df.iloc[0]
    row2 = df.iloc[1]
    row3 = df.iloc[2]
    row4 = df.iloc[3]
    df = df.drop(df.index[-1])


    def combine_rows(row1, row2):
        combined_row = {}
        for col in row1.index:
            combined_row[col] = (str(row1[col]) + " " + str(row2[col])).strip()
        return pd.DataFrame([combined_row]).iloc[0]  
        

    header_row = combine_rows(combine_rows(combine_rows(row1, row2), row3), row4)

    #Set top row as header row values
    df.columns = header_row.tolist()
    df = df.drop(df.index[[0,1,2,3]])  # Drop the first four rows which are now redundant

    df = df.reset_index(drop=True)  # Reset the index

    #Convert empty strings to 0
    df = df.replace(" ", np.nan)

    #convert data types
    df["Date"] = pd.to_datetime(df["Date"], format = "%d/%m/%Y")

    return df



In [4]:
def insert_data_from_file(file_path, connection, db_table):
    try:
        df = extract_data(file_path)
    except Exception as e:
        return e

    try:
        df.to_sql(db_table, connection, if_exists='append', index=False)   
    except Exception as e:
        return e

In [ ]:
# # Define suffixes to try
# SUFFIXES = [" AWS", " AIRPORT", " STATION"]

# def find_station_name(station_name, conn, station_table="stations"):
#     """Try exact + suffix matches for a station name."""
#     station_name = station_name.strip()
#     candidates = [station_name] + [station_name + s for s in SUFFIXES]


#     for candidate in candidates:
#         exists = pd.read_sql_query(
#             sqlalchemy.text(f"SELECT 1 FROM {station_table} WHERE `Station Name` = '{candidate}' LIMIT 1;"),
#             conn,
            
#         )
#         if not exists.empty:
#             return candidate  # found a match

#     return None  # nothing matched

In [5]:
# Define suffixes to try
SUFFIXES = [" AWS", " AIRPORT", " STATION"]

def find_station_name(station_name, conn, station_table="stations"):
    """Try exact + suffix matches for a station name."""
    station_name = station_name.strip()
    candidates = [station_name] + [station_name + s for s in SUFFIXES]


    for candidate in candidates:
        exists = pd.read_sql_query(
            sqlalchemy.text(f"SELECT 1 FROM {station_table} WHERE `Station Name` = '{candidate}' LIMIT 1;"),
            conn,
            
        )
        if not exists.empty:
            return candidate  # found a match

    return None  # nothing matched

<h3>Establish file paths & connections</h3>

In [6]:
#Connection to database
def connect_with_connector() -> sqlalchemy.engine.base.Engine:
    """
    Initializes a connection pool for a Cloud SQL instance of MySQL.

    Uses the Cloud SQL Python Connector package.
    """

    instance_connection_name = "esoteric-life-470902-k9:australia-southeast2:jyjam-weatherdata-db"
    db_user = "admin" 
    db_pass = "Password1!"
    db_name ="weather_data_database"

    ip_type = IPTypes.PRIVATE if os.environ.get("PRIVATE_IP") else IPTypes.PUBLIC

    # initialize Cloud SQL Python Connector object
    connector = Connector(ip_type=ip_type, refresh_strategy="LAZY")

    def getconn() -> pymysql.connections.Connection:
        conn: pymysql.connections.Connection = connector.connect(
            instance_connection_name,
            "pymysql",
            user=db_user,
            password=db_pass,
            db=db_name,
        )
        return conn

    pool = sqlalchemy.create_engine(
        "mysql+pymysql://",
        creator=getconn,
    )
    return pool


In [7]:
db_conn = connect_with_connector()
db_table = "weather_data"

In [12]:
import pandas as pd
import sqlalchemy

# Test retrieval
def test_connection():
    try:
        # Get the engine from your function
        engine = connect_with_connector()

        # Replace with an actual table name in your DB
        db_table = db_table

        # Run a simple query
        query = f"SELECT * FROM {db_table} LIMIT 5;"
        df = pd.read_sql(query, con=engine)

        print("✅ Connection successful, retrieved data:")
        print(df)

    except Exception as e:
        print("❌ Error testing connection:", e)

# Example run
if __name__ == "__main__":
    test_connection()


❌ Error testing connection: cannot access local variable 'db_table' where it is not associated with a value


<h3>RESET TABLE ROWS</h3>

In [ ]:
# SQL statement to delete all rows from the table
# sql_delete_all_rows = f"DELETE FROM {db_table};"



# with db_conn.connect() as cursor:
#     try:
#         # Execute the DELETE statement
#         cursor.execute(sqlalchemy.text(sql_delete_all_rows))

#         # Commit the changes to the database
#         cursor.commit()

#         print(f"All rows deleted from table '{db_table}'.")

#     except Exception as e:
#         print(f"Error deleting rows: {e}")

#     finally:
#         # Close the cursor
#         cursor.close()


All rows deleted from table 'weather_data'.


<h3>MAKE UNIQUE TABLE DATA</h3>

In [ ]:
# cursor = db_conn.cursor()


# sql_create_unique_table = "CREATE TABLE weather_data_temp AS SELECT DISTINCT * FROM weather_data"
# sql_drop_old_table = "DROP TABLE weather_data"
# sql_create_new_table = "CREATE TABLE weather_data AS SELECT DISTINCT * FROM weather_data_temp"
# sql_drop_temp_table = "DROP TABLE weather_data_temp"

# try:
#     # Execute the DELETE statement
#     cursor.execute(sql_create_unique_table)
#     cursor.execute(sql_drop_old_table)
#     cursor.execute(sql_create_new_table)
#     cursor.execute(sql_drop_temp_table)


#     # Commit the changes to the database
#     db_conn.commit()

#     print(f"Distinct '{db_table}' table has been created.")

# except sqlite3.Error as e:
#     print(f"Error creating distinct table: {e}")

# finally:
#     # Close the cursor
#     cursor.close()

<h3>INSERT ALL Data into weather_data table</h3>

In [ ]:

zip_file_path = './../Data/WeatherDownload.zip'
target_file_in_zip = 'WeatherDownload/vic/aireys_inlet-201508.csv'



try:
    with ZipFile(zip_file_path, 'r') as zf:
        members = zf.infolist()

        with db_conn.connect() as conn:
            # Load existing data from the table
            existing_df = pd.read_sql_table("stations", conn)

            for member in members: #get each item in directory
                if not member.is_dir(): #if item is not directory. i.e. item is a file
                    with zf.open(member.filename, 'r') as file: #open file using filename
                        pass
                        try:
                            df = extract_data(file) #return dataframe created from file
                            station_name = df["Station Name"].iloc[0]

                        
                            # # get the station name from df
                            matched_name = find_station_name(station_name, conn)
                        
                            
                            if matched_name:
                                df["Station Name"] = matched_name  # update all rows
                                try:
                                    df.to_sql("weather_data", db_conn, if_exists="append", index=False)
                                    print(f"✅ Inserted '{member.filename}' as '{matched_name}'")
                                except Exception as e:
                                    print("❌ Error inserting file:", member.filename, e)
                            else:
                                print(f"⚠️ Skipping {member.filename}: no match for {station_name}")

                        except Exception as e:
                            print("Error extracting data:", e)

                        # matched_name = existing_df[station_name]
                        # print(matched_name)


                        # try: 
                        #     df = extract_data(file) 
                        # except Exception as e: 
                        #     print("Error extracting data:", e) 
                        #     # 'if_exists' can be 'fail', 'replace', or 'append' 
                        #     # 'index=False' prevents writing the DataFrame index as a column in the database 
                        #     try: 
                        #         df.to_sql(db_table, conn, if_exists='append', index=False) 
                        #         print(f"'{member.filename}' was inserted successfully") 
                        #     except Exception as e: 
                        #         print("error with file:", file, "ERROR CAUGHT:", e) 
                        #         pass


            conn.close()



except FileNotFoundError:
    print(f"Error: ZIP file '{zip_file_path}' not found.")
except KeyError:
    print(f"Error: File '{target_file_in_zip}' not found in '{zip_file_path}'.")
except Exception as e:
    print(f"An error occurred: {e}")
    

Error extracting data: ('ERROR READING CSV:', <zipfile.ZipExtFile name='WeatherDownload/nsw/albion_park_(shellharbour_airport)-201508.csv' mode='r' compress_type=deflate>, '-- WITH ERROR:', EmptyDataError('No columns to parse from file'))
Error extracting data: ('ERROR READING CSV:', <zipfile.ZipExtFile name='WeatherDownload/nsw/albion_park_(shellharbour_airport)-201509.csv' mode='r' compress_type=deflate>, '-- WITH ERROR:', EmptyDataError('No columns to parse from file'))
Error extracting data: ('ERROR READING CSV:', <zipfile.ZipExtFile name='WeatherDownload/nsw/albion_park_(shellharbour_airport)-201510.csv' mode='r' compress_type=deflate>, '-- WITH ERROR:', EmptyDataError('No columns to parse from file'))
Error extracting data: ('ERROR READING CSV:', <zipfile.ZipExtFile name='WeatherDownload/nsw/albion_park_(shellharbour_airport)-201511.csv' mode='r' compress_type=deflate>, '-- WITH ERROR:', EmptyDataError('No columns to parse from file'))
Error extracting data: ('ERROR READING CSV:'